In [1]:
# Starting point of program execution.
## The application receives audio/speech input and generates image as the ouput on the gradio webapp


import torch
import warnings
import gradio as gr
import numpy as np
from IPython.display import display
from transformers import pipeline
print("Successfully imported all the libraries")

Successfully imported all the libraries


In [2]:
#Invoke the Image generator function to convert text to speech
%run image_generator.ipynb

In [3]:
warnings.filterwarnings('ignore')
# Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load the ASR pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    #model="openai/whisper-large",
    model="openai/whisper-small",
    chunk_length_s=30,
    device=device,
)

In [4]:
def transcribe(audio):
    try:
        sr, y = audio
    
        # Convert to mono if stereo
        if y.ndim > 1:
            y = y.mean(axis=1)
        
        y = y.astype(np.float32)
        y /= np.max(np.abs(y))

        result_text = pipe({"sampling_rate": sr, "raw": y})["text"]
        #print(result_text)
        image = image_creation(result_text)
        return result_text, image
        return result_text
    except Exception as e:
        return str(e)

In [5]:
demo = gr.Interface(
    transcribe,
    #gr.Audio(sources="microphone"),
    inputs=gr.Audio(type="numpy"),
    #outputs="text",
    outputs=[gr.Textbox(label="Transcription"),gr.Image(label="Image")],
    title="Speech to Image Conversion Application",
)

In [6]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e73b2a52d5a40de560.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
